In [7]:
import numpy as np
import pandas as pd
import urllib
from sodapy import Socrata

In [8]:
client = Socrata("data.montgomeryal.gov", None)
results = client.get("pjb8-sd6v", limit=10000)
raw_data = pd.DataFrame.from_records(results)

In [9]:
#create copy of raw_data as data
data = raw_data.sort_index()
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2405 entries, 0 to 2404
Data columns (total 12 columns):
annual_salary      2405 non-null object
annualsalaryytd    2400 non-null object
department         2405 non-null object
grade              2405 non-null object
hire_date          2405 non-null object
name               2405 non-null object
otherpayamt        212 non-null object
otherpaydesc       1313 non-null object
overtimeamt        2311 non-null object
position_title     2405 non-null object
positiontype       2405 non-null object
step               2405 non-null object
dtypes: object(12)
memory usage: 225.5+ KB


In [10]:
#convert columns to appropriate types, data cleanup, and reset the index
cat_cols = pd.Index(['department', 'otherpaydesc', 'positiontype'])
#data[cat_cols] = data[cat_cols].apply(pd.Categorical)

data['otherpayamt'] = data['otherpayamt'].fillna(0) #replace null values with 0s
data['overtimeamt'] = data['overtimeamt'].fillna(0) #replace null values with 0s
data['hire_date'] = pd.to_datetime(data['hire_date'])

num_cols = pd.Index(['annual_salary', 'annualsalaryytd', 'otherpayamt', 'overtimeamt', 'step'])
data[num_cols] = data[num_cols].apply(pd.to_numeric, errors='coerce')

#data.set_index('hire_date', drop=False, inplace=True)

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2405 entries, 0 to 2404
Data columns (total 12 columns):
annual_salary      2405 non-null float64
annualsalaryytd    2400 non-null float64
department         2405 non-null object
grade              2405 non-null object
hire_date          2405 non-null datetime64[ns]
name               2405 non-null object
otherpayamt        2405 non-null float64
otherpaydesc       1313 non-null object
overtimeamt        2405 non-null float64
position_title     2405 non-null object
positiontype       2405 non-null object
step               2405 non-null int64
dtypes: datetime64[ns](1), float64(4), int64(1), object(6)
memory usage: 225.5+ KB


In [11]:
#create dictionary for departments by type and affix to dataframe
positions = data.department.unique()
types = ['Public Works', 'Public Safety', 'Maintenance/Sanitation', 
         'Public Works', 'Public Works', 'Maintenance/Sanitation', 
         'Public Safety', 'Public Safety', 'Public Works', 'Public Works', 
         'Admin & Judicial', 'Maintenance/Sanitation', 'Public Works', 
         'Admin & Judicial', 'Maintenance/Sanitation',  'Admin & Judicial', 'Admin & Judicial',
         'Maintenance/Sanitation', 'Public Safety', 'Admin & Judicial', 
         'Public Works', 'Admin & Judicial', 'Admin & Judicial', 
         'Admin & Judicial', 'Public Safety', 'Public Safety', 
         'Admin & Judicial', 'Admin & Judicial', 'Admin & Judicial', 
         'Admin & Judicial', 'Public Safety']
gov_dict = dict(zip(positions, types))
data['category'] = data['department'].map(gov_dict)

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import Imputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import FeatureUnion

# Create a class to select numerical or categorical columns 
# since Scikit-Learn doesn't handle DataFrames yet
class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.attribute_names].values

In [14]:
# Definition of the CategoricalEncoder class, copied from PR #9151.
# Just run this cell, or copy it to your code, do not try to understand it (yet).

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.utils import check_array
from sklearn.preprocessing import LabelEncoder
from scipy import sparse

class CategoricalEncoder(BaseEstimator, TransformerMixin):
    """Encode categorical features as a numeric array.
    The input to this transformer should be a matrix of integers or strings,
    denoting the values taken on by categorical (discrete) features.
    The features can be encoded using a one-hot aka one-of-K scheme
    (``encoding='onehot'``, the default) or converted to ordinal integers
    (``encoding='ordinal'``).
    This encoding is needed for feeding categorical data to many scikit-learn
    estimators, notably linear models and SVMs with the standard kernels.
    Read more in the :ref:`User Guide <preprocessing_categorical_features>`.
    Parameters
    ----------
    encoding : str, 'onehot', 'onehot-dense' or 'ordinal'
        The type of encoding to use (default is 'onehot'):
        - 'onehot': encode the features using a one-hot aka one-of-K scheme
          (or also called 'dummy' encoding). This creates a binary column for
          each category and returns a sparse matrix.
        - 'onehot-dense': the same as 'onehot' but returns a dense array
          instead of a sparse matrix.
        - 'ordinal': encode the features as ordinal integers. This results in
          a single column of integers (0 to n_categories - 1) per feature.
    categories : 'auto' or a list of lists/arrays of values.
        Categories (unique values) per feature:
        - 'auto' : Determine categories automatically from the training data.
        - list : ``categories[i]`` holds the categories expected in the ith
          column. The passed categories are sorted before encoding the data
          (used categories can be found in the ``categories_`` attribute).
    dtype : number type, default np.float64
        Desired dtype of output.
    handle_unknown : 'error' (default) or 'ignore'
        Whether to raise an error or ignore if a unknown categorical feature is
        present during transform (default is to raise). When this is parameter
        is set to 'ignore' and an unknown category is encountered during
        transform, the resulting one-hot encoded columns for this feature
        will be all zeros.
        Ignoring unknown categories is not supported for
        ``encoding='ordinal'``.
    Attributes
    ----------
    categories_ : list of arrays
        The categories of each feature determined during fitting. When
        categories were specified manually, this holds the sorted categories
        (in order corresponding with output of `transform`).
    Examples
    --------
    Given a dataset with three features and two samples, we let the encoder
    find the maximum value per feature and transform the data to a binary
    one-hot encoding.
    >>> from sklearn.preprocessing import CategoricalEncoder
    >>> enc = CategoricalEncoder(handle_unknown='ignore')
    >>> enc.fit([[0, 0, 3], [1, 1, 0], [0, 2, 1], [1, 0, 2]])
    ... # doctest: +ELLIPSIS
    CategoricalEncoder(categories='auto', dtype=<... 'numpy.float64'>,
              encoding='onehot', handle_unknown='ignore')
    >>> enc.transform([[0, 1, 1], [1, 0, 4]]).toarray()
    array([[ 1.,  0.,  0.,  1.,  0.,  0.,  1.,  0.,  0.],
           [ 0.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.]])
    See also
    --------
    sklearn.preprocessing.OneHotEncoder : performs a one-hot encoding of
      integer ordinal features. The ``OneHotEncoder assumes`` that input
      features take on values in the range ``[0, max(feature)]`` instead of
      using the unique values.
    sklearn.feature_extraction.DictVectorizer : performs a one-hot encoding of
      dictionary items (also handles string-valued features).
    sklearn.feature_extraction.FeatureHasher : performs an approximate one-hot
      encoding of dictionary items or strings.
    """

    def __init__(self, encoding='onehot', categories='auto', dtype=np.float64,
                 handle_unknown='error'):
        self.encoding = encoding
        self.categories = categories
        self.dtype = dtype
        self.handle_unknown = handle_unknown

    def fit(self, X, y=None):
        """Fit the CategoricalEncoder to X.
        Parameters
        ----------
        X : array-like, shape [n_samples, n_feature]
            The data to determine the categories of each feature.
        Returns
        -------
        self
        """

        if self.encoding not in ['onehot', 'onehot-dense', 'ordinal']:
            template = ("encoding should be either 'onehot', 'onehot-dense' "
                        "or 'ordinal', got %s")
            raise ValueError(template % self.handle_unknown)

        if self.handle_unknown not in ['error', 'ignore']:
            template = ("handle_unknown should be either 'error' or "
                        "'ignore', got %s")
            raise ValueError(template % self.handle_unknown)

        if self.encoding == 'ordinal' and self.handle_unknown == 'ignore':
            raise ValueError("handle_unknown='ignore' is not supported for"
                             " encoding='ordinal'")

        X = check_array(X, dtype=np.object, accept_sparse='csc', copy=True)
        n_samples, n_features = X.shape

        self._label_encoders_ = [LabelEncoder() for _ in range(n_features)]

        for i in range(n_features):
            le = self._label_encoders_[i]
            Xi = X[:, i]
            if self.categories == 'auto':
                le.fit(Xi)
            else:
                valid_mask = np.in1d(Xi, self.categories[i])
                if not np.all(valid_mask):
                    if self.handle_unknown == 'error':
                        diff = np.unique(Xi[~valid_mask])
                        msg = ("Found unknown categories {0} in column {1}"
                               " during fit".format(diff, i))
                        raise ValueError(msg)
                le.classes_ = np.array(np.sort(self.categories[i]))

        self.categories_ = [le.classes_ for le in self._label_encoders_]

        return self

    def transform(self, X):
        """Transform X using one-hot encoding.
        Parameters
        ----------
        X : array-like, shape [n_samples, n_features]
            The data to encode.
        Returns
        -------
        X_out : sparse matrix or a 2-d array
            Transformed input.
        """
        X = check_array(X, accept_sparse='csc', dtype=np.object, copy=True)
        n_samples, n_features = X.shape
        X_int = np.zeros_like(X, dtype=np.int)
        X_mask = np.ones_like(X, dtype=np.bool)

        for i in range(n_features):
            valid_mask = np.in1d(X[:, i], self.categories_[i])

            if not np.all(valid_mask):
                if self.handle_unknown == 'error':
                    diff = np.unique(X[~valid_mask, i])
                    msg = ("Found unknown categories {0} in column {1}"
                           " during transform".format(diff, i))
                    raise ValueError(msg)
                else:
                    # Set the problematic rows to an acceptable value and
                    # continue `The rows are marked `X_mask` and will be
                    # removed later.
                    X_mask[:, i] = valid_mask
                    X[:, i][~valid_mask] = self.categories_[i][0]
            X_int[:, i] = self._label_encoders_[i].transform(X[:, i])

        if self.encoding == 'ordinal':
            return X_int.astype(self.dtype, copy=False)

        mask = X_mask.ravel()
        n_values = [cats.shape[0] for cats in self.categories_]
        n_values = np.array([0] + n_values)
        indices = np.cumsum(n_values)

        column_indices = (X_int + indices[:-1]).ravel()[mask]
        row_indices = np.repeat(np.arange(n_samples, dtype=np.int32),
                                n_features)[mask]
        data = np.ones(n_samples * n_features)[mask]

        out = sparse.csc_matrix((data, (row_indices, column_indices)),
                                shape=(n_samples, indices[-1]),
                                dtype=self.dtype).tocsr()
        if self.encoding == 'onehot-dense':
            return out.toarray()
        else:
            return out

In [15]:
num_attribs = ["step"]
cat_attribs = ["department", "grade", "position_title", "positiontype", "category"]

num_pipeline = Pipeline([
        ('selector', DataFrameSelector(num_attribs)),
        ('imputer', Imputer(strategy="median")),
        ('std_scaler', StandardScaler()),
    ])

cat_pipeline = Pipeline([
        ('selector', DataFrameSelector(cat_attribs)),
        ('cat_encoder', CategoricalEncoder(encoding="onehot-dense")),
    ])

full_pipeline = FeatureUnion(transformer_list=[
        ("num_pipeline", num_pipeline),
        ("cat_pipeline", cat_pipeline),
    ])

In [16]:
X = data.drop(['annualsalaryytd', 'name', 'otherpayamt', 'otherpaydesc', 'overtimeamt', 'hire_date', 'annual_salary'], axis=1)
y = data[["annual_salary"]]

In [18]:
X_transformed = full_pipeline.fit_transform(X)

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X_transformed, y, test_size=0.20, random_state=42)

In [20]:
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train.values)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [21]:
from sklearn.metrics import mean_squared_error
lin_mse = mean_squared_error(y_train, lin_reg.predict(X_train))
lin_rmse = np.sqrt(lin_mse)
lin_rmse

1003.2107461290498

In [51]:
lin_mse = mean_squared_error(y_test, lin_reg.predict(X_test))
lin_rmse = np.sqrt(lin_mse)
lin_rmse

236734238916901.53

In [43]:
predictions_train = lin_reg.predict(X_train)
print("r2/variance : ", lin_reg.score(X_train, y_train))
print("Residual sum of squares: %.2f"
              % np.mean((lin_reg.predict(X_train) - y_train) ** 2))

r2/variance :  0.995621044238
Residual sum of squares: 1006431.80


In [34]:
predictions = lin_reg.predict(X_test)
print("r2/variance : ", lin_reg.score(X_test, y_test))
print("Residual sum of squares: %.2f"
              % np.mean((lin_reg.predict(X_test) - y_test) ** 2))

r2/variance :  -2.52821285125e+20
Residual sum of squares: 56043099875564615923895631872.00


In [27]:
X.iloc[17]

department            FIRE DEPARTMENT
grade                             PSE
position_title    DISTRICT FIRE CHIEF
positiontype         Full Time Exempt
step                                5
category                Public Safety
Name: 17, dtype: object

In [28]:
pred = pd.DataFrame(np.int64(lin_reg.predict(X_test)))
pred.iloc[17]

0    3648648306793169
Name: 17, dtype: int64

In [29]:
data.iloc[218]

annual_salary                  50251.8
annualsalaryytd                5798.29
department          CITY INVESTIGATION
grade                              A08
hire_date          2017-02-24 00:00:00
name               BAROUSSE, WILLIAM M
otherpayamt                          0
otherpaydesc                       NaN
overtimeamt                          0
position_title       CITY INVESTIGATOR
positiontype                 Full Time
step                                 4
category                 Public Safety
Name: 218, dtype: object

In [30]:
X.iloc[217:220]

,department,grade,position_title,positiontype,step,category
217,LIBRARY,A02,LIBRARY ASSISTANT I,Full Time,2,Public Works
218,CITY INVESTIGATION,A08,CITY INVESTIGATOR,Full Time,4,Public Safety
219,FIRE DEPARTMENT,PSC,FIRE LIEUTENANT,Full Time,5,Public Safety


In [31]:
np.int64(predictions[:18,:])

array([[           47000],
       [           42786],
       [           51700],
       [           38750],
       [           32328],
       [           40764],
       [           57344],
       [           35000],
       [           40956],
       [           65972],
       [           17996],
       [           55156],
       [           32328],
       [           34350],
       [           39752],
       [           39752],
       [           30862],
       [3648648306793169]], dtype=int64)

In [32]:
y_test.annual_salary[:18]

2179    46946.39
1525    43117.57
1377    50968.74
2372    38316.10
1125    32318.00
1697    40676.69
1973    57911.24
1027    34567.10
929     40894.26
210     63243.98
1517    18000.00
1724    54436.30
927     32318.00
2028    34599.34
1908    39456.35
56      39456.35
1938    29955.95
218     50251.76
Name: annual_salary, dtype: float64

In [50]:
np.int64(predictions_train[predictions_train > 155000])

array([], dtype=int64)

In [37]:
predictions.min()

-241517544334976.0

In [66]:
from sklearn.decomposition import PCA

In [68]:
pca = PCA()
x2d = pca.fit_transform(X_transformed)
cumsum = np.cumsum(pca.explained_variance_ratio_)
d = np.argmax(cumsum >= .95) + 1

In [72]:
pca = PCA(n_components = 0.95)
x2d = pca.fit_transform(X_transformed)

In [73]:
pca.explained_variance_ratio_

array([ 0.21619108,  0.1194362 ,  0.07915185,  0.06953584,  0.05091849,
        0.03200067,  0.02886885,  0.02676108,  0.02359933,  0.02018242,
        0.01715609,  0.01576915,  0.0154829 ,  0.01474405,  0.01316522,
        0.01124292,  0.01045958,  0.01027488,  0.00919351,  0.0085542 ,
        0.00847718,  0.00754848,  0.00710985,  0.00670706,  0.00602174,
        0.00596816,  0.00571172,  0.00543932,  0.00505665,  0.00475863,
        0.00454294,  0.00428461,  0.00413338,  0.0040096 ,  0.00387841,
        0.0036866 ,  0.00355191,  0.00336307,  0.0031263 ,  0.00307894,
        0.00303144,  0.0029289 ,  0.00268351,  0.00262569,  0.00257901,
        0.00250444,  0.00242037,  0.00231523,  0.00224006,  0.00210778,
        0.00198069,  0.00189523,  0.00177789,  0.00170691,  0.00164992,
        0.00161557,  0.00153785,  0.00148242,  0.00147   ,  0.00133834,
        0.00122551,  0.00120655,  0.00117313,  0.0011331 ,  0.00106074,
        0.0010207 ,  0.00097958,  0.00092351,  0.00087715,  0.00

In [1]:
!python 3 -c 'import tensorflow; print(tensorflow.__version__)'

python: can't open file '3': [Errno 2] No such file or directory


In [3]:
!pip install --upgrade setuptools

  Found existing installation: setuptools 36.5.0.post20170921
    Uninstalling setuptools-36.5.0.post20170921:
      Successfully uninstalled setuptools-36.5.0.post20170921


In [7]:
!easy_install -U setuptools

Searching for setuptools
Reading https://pypi.python.org/simple/setuptools/
Best match: setuptools 38.5.2
Processing setuptools-38.5.2-py2.py3-none-any.whl
Installing setuptools-38.5.2-py2.py3-none-any.whl to c:\users\595217\appdata\local\continuum\anaconda3\lib\site-packages
writing requirements to c:\users\595217\appdata\local\continuum\anaconda3\lib\site-packages\setuptools-38.5.2-py3.6.egg\EGG-INFO\requires.txt
Adding setuptools 38.5.2 to easy-install.pth file
Installing easy_install-script.py script to c:\users\595217\appdata\local\continuum\anaconda3\Scripts
Installing easy_install.exe script to c:\users\595217\appdata\local\continuum\anaconda3\Scripts


c:\users\595217\appdata\local\continuum\anaconda3\lib\site-packages\setuptools\pep425tags.py:89: RuntimeWarning: Config variable 'Py_DEBUG' is unset, Python ABI tag may be incorrect
  warn=(impl == 'cp')):
c:\users\595217\appdata\local\continuum\anaconda3\lib\site-packages\setuptools\pep425tags.py:93: RuntimeWarning: Config variable 'WITH_PYMALLOC' is unset, Python ABI tag may be incorrect
  warn=(impl == 'cp')):
error: [WinError 5] Access is denied: 'c:\\users\\595217\\appdata\\local\\continuum\\anaconda3\\Scripts\\easy_install.exe'


In [6]:
!pip install distribute

  Using cached distribute-0.7.3.zip
    Complete output from command python setup.py egg_info:
    Traceback (most recent call last):
      File "<string>", line 1, in <module>
      File "C:\Users\595217\AppData\Local\Temp\pip-build-4dw77r2w\distribute\setuptools\__init__.py", line 2, in <module>
        from setuptools.extension import Extension, Library
      File "C:\Users\595217\AppData\Local\Temp\pip-build-4dw77r2w\distribute\setuptools\extension.py", line 5, in <module>
        from setuptools.dist import _get_unpatched
      File "C:\Users\595217\AppData\Local\Temp\pip-build-4dw77r2w\distribute\setuptools\dist.py", line 7, in <module>
        from setuptools.command.install import install
      File "C:\Users\595217\AppData\Local\Temp\pip-build-4dw77r2w\distribute\setuptools\command\__init__.py", line 8, in <module>
        from setuptools.command import install_scripts
      File "C:\Users\595217\AppData\Local\Temp\pip-build-4dw77r2w\distribute\setuptools\command\install_scrip

Command "python setup.py egg_info" failed with error code 1 in C:\Users\595217\AppData\Local\Temp\pip-build-4dw77r2w\distribute\


In [8]:
!sudo apt-get install python-setuptools

'sudo' is not recognized as an internal or external command,
operable program or batch file.
